
# Nutrition5k — Single Image Test (YOLO11n‑cls)

This loads Nutrition5k `best.pt`** (runs named `nutrition5k_yolo11n_cls_*`), classifies a single image,
prints **Top‑1 / Top‑5**, and saves an annotated image.


In [1]:

IMAGE_PATH = "/home/kristoffel/utilities/util_test_images/pasta-carbonara.jpg"  # ← change to any image to test
MODEL_DIR  = "/home/kristoffel/models"
RUN_PREFIX = "nutrition5k_yolo11n_cls_"

import os, glob
assert os.path.isfile(IMAGE_PATH), f"Image not found: {IMAGE_PATH}"
print("Image:", IMAGE_PATH)


Image: /home/kristoffel/utilities/util_test_images/pasta-carbonara.jpg


In [2]:

# Find newest best.pt
import os, glob
cands = glob.glob(os.path.join(MODEL_DIR, RUN_PREFIX + "*", "weights", "best.pt"))
assert cands, f"No best.pt found matching {MODEL_DIR}/{RUN_PREFIX}*/weights/best.pt"
BEST_WEIGHTS = max(cands, key=os.path.getmtime)
print("Using:", BEST_WEIGHTS)


AssertionError: No best.pt found matching /home/kristoffel/models/nutrition5k_yolo11n_cls_*/weights/best.pt

In [ ]:

# Predict
# !pip install -U ultralytics
import torch
from ultralytics import YOLO

device = 0 if torch.cuda.is_available() else "cpu"
model = YOLO(BEST_WEIGHTS)
results = model.predict(
    source=IMAGE_PATH,
    imgsz=224,
    device=device,
    save=True,
    project=MODEL_DIR,
    name="nutrition5k_single_inference"
)
print("Saved annotated image to:", os.path.join(MODEL_DIR, "nutrition5k_single_inference"))


In [ ]:

# Show Top-1 / Top-5
import torch as th
import pandas as pd
from IPython.display import display, Image as IPyImage
import os

r = results[0]
names = model.names
probs = r.probs.data if hasattr(r.probs, "data") else r.probs

topk = min(5, probs.shape[-1])
conf, idx = th.topk(probs, k=topk)
idx = idx.tolist(); conf = conf.tolist()

print(f"Top-1: {names[idx[0]]} (confidence={float(conf[0]):.4f})")
df = pd.DataFrame({
    "rank": list(range(1, topk+1)),
    "class": [names[i] for i in idx],
    "confidence": [float(c) for c in conf]
})
display(df)

annot_path = os.path.join(MODEL_DIR, "nutrition5k_single_inference", os.path.basename(IMAGE_PATH))
if os.path.exists(annot_path):
    display(IPyImage(filename=annot_path))
